In [ ]:
%pip install transformers
%pip install datasets
%pip install evaluate
%pip install sklearn

In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files={"train": "hf_train.csv", "test": "hf_test.csv"})
dataset

In [ ]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)

from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', max_length=512)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
from transformers import RobertaForSequenceClassification
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", 
                                  num_train_epochs=10, 
                                  learning_rate=1e-5, 
                                  per_device_train_batch_size=16,
                                  logging_strategy = 'epoch',
                                  logging_first_step = True,
                                  save_strategy = 'epoch',
                                  load_best_model_at_end = True,
                                  metric_for_best_model = 'f1'
                                  )

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("f1")
# metric_2 = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='macro')

# def compute_metrics(eval_pred):
#   metric = evaluate.load("f1")
#   metric_2 = evaluate.load("accuracy")
#   logits, labels = eval_pred
#   predictions = np.argmax(logits, axis=-1)
#   f1 = metric.compute(predictions=predictions, references=labels, average='macro')
#   accuracy = metric2.compute(predictions=predictions, references=labels)
#   return {"f1": f1, "accuracy": accuracy}

In [ ]:
train_dataset = tokenized_datasets["train"].shuffle(seed=41).select(range(len(tokenized_datasets["train"])))
test_dataset = tokenized_datasets["test"].shuffle(seed=41).select(range(len(tokenized_datasets["test"])))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
# t = trainer.predict(test_dataset)

In [ ]:
# metric = evaluate.load("f1")

# logits = t.predictions
# predictions = np.argmax(logits, axis=-1)
# labels = t.label_ids
# metric.compute(predictions=predictions, references=labels, average='macro')